In [1]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving cache.py to cache.py
Saving cifar10.py to cifar10.py
Saving dataset.py to dataset.py
Saving download.py to download.py


In [0]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import time
import cifar10

In [0]:
cifar10.data_path = "data/CIFAR-10/"

In [4]:
cifar10.maybe_download_and_extract()

- Download progress: 100.0%
Download finished. Extracting files.
Done.


In [5]:
class_names = cifar10.load_class_names()
class_names

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [11]:
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [12]:
print(cls_train.shape)
print(labels_train.shape)
print(images_train.shape)
print(images_test.shape)

(50000,)
(50000, 10)
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [0]:
#xtrain=images_train.reshape(50000,3072)

In [9]:
import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense,Dropout

Using TensorFlow backend.


In [0]:
input_width=32
input_height=32
input_channels=3
input_pixels=3072

n_conv1=32
n_conv2=64
stride_conv1=1
stride_conv2=1
conv1_k=5
conv2_k=5
max_pool1_k=2
max_pool2_k=2

n_hidden=1024
n_out=10

input_size_to_hidden=(input_width//(max_pool1_k*max_pool2_k))*(input_height//(max_pool1_k*max_pool2_k))*n_conv2


In [0]:
model=Sequential()
model.add(Conv2D(filters=n_conv1,kernel_size=(conv1_k,conv1_k),strides=(stride_conv1,stride_conv1),padding='same',input_shape=(input_width,input_height,input_channels),activation='relu')) 
model.add(MaxPooling2D(pool_size=(max_pool1_k,max_pool1_k),padding='same'))

#model.add(Convolution2D(n_conv2,conv2_k,conv2_k))
model.add(Conv2D(filters=n_conv2,kernel_size=(conv2_k,conv2_k),strides=(stride_conv2,stride_conv2),padding='same',activation='relu')) 
#model.add(Conv2D(filters=n_conv2,kernel_size=conv2_k,strides=stride_conv2,padding='same',activation='relu')) 

model.add(MaxPooling2D(pool_size=(max_pool2_k,max_pool2_k),padding='same'))
model.add(Flatten())
model.add(Dense(units=1024,activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(units=10,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [15]:
model.fit(images_train,labels_train,epochs=25,batch_size=100,validation_data=(images_test,labels_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 14s 273us/step - loss: 1.6116 - acc: 0.4114 - val_loss: 1.2700 - val_acc: 0.5534
Epoch 2/25
50000/50000 [==============================] - 11s 225us/step - loss: 1.2511 - acc: 0.5564 - val_loss: 1.0566 - val_acc: 0.6326
Epoch 3/25
19600/50000 [==========>...................] - ETA: 6s - loss: 1.1337 - acc: 0.599050000/50000 [==============================] - 11s 225us/step - loss: 1.0963 - acc: 0.6132 - val_loss: 0.9672 - val_acc: 0.6660
Epoch 4/25
50000/50000 [==============================] - 11s 225us/step - loss: 0.9952 - acc: 0.6528 - val_loss: 0.9097 - val_acc: 0.6828
Epoch 5/25
44500/50000 [=========================>....] - ETA: 1s - loss: 0.9166 - acc: 0.678950000/50000 [==============================] - 11s 226us/step - loss: 0.9165 - acc: 0.6789 - val_loss: 0.8678 - val_acc: 0.6972
Epoch 6/25
50000/50000 [==============================] - 11s 225us/step - loss: 0.8482 -

In [16]:
preds=model.predict(images_test)
preds.shape

(10000, 10)

In [17]:
score=model.evaluate(images_test,labels_test)
score

10000/10000 [==============================] - 2s 160us/step


[0.963775114774704, 0.7413]

In [0]:
predictions=[]

In [0]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())
predictions=tf.argmax(preds,1)

In [27]:
arr=sess.run(predictions)
arr

array([3, 8, 8, ..., 5, 4, 7])

In [28]:
prednames=[class_names[i] for i in arr]
prednames[:5]

['cat', 'ship', 'ship', 'airplane', 'deer']

In [29]:
predsave2=np.array(prednames)
predsave2[1:5]

array(['ship', 'ship', 'airplane', 'deer'], dtype='<U10')

In [0]:
np.savetxt('savedfileCNN.csv',predsave2,fmt='%s')

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import io
import zipfile
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
uploadId = '1IZfLl2tUs3o9maAC2rBPqVuwmS2fyY-F'
uploaded = drive.CreateFile({'parents':[{'id': uploadId}] , 'title' : 'cifarpredictionsCNN'})
uploaded.SetContentFile('savedfileCNN.csv')
uploaded.Upload()
                #dataName=chkName+".data-00000-of-00001"
                #check="data/"+dataName
                

In [0]:
filecsv=pd.read_csv('savedfileCNN.csv')

In [37]:
filecsv

,cat
0,ship
1,ship
2,airplane
3,deer
4,frog
5,automobile
6,deer
7,cat
8,automobile
9,airplane


In [0]:
uploaded = drive.CreateFile({'title': 'sample.csv', 'mimeType': 'text/csv'})

NameError: ignored

In [0]:
labels_train[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [0]:
cls_train[1]

9